In [1]:
import pandapower as pp 
import numpy as np
import pandas as pd
import os
import random
import pandapower as pp
from pandapower.timeseries import DFData
from pandapower.timeseries import OutputWriter
from pandapower.timeseries.run_time_series import run_timeseries
from pandapower.control import ConstControl
from shapely.geometry import Point
from pandapower.estimation import estimate

In [2]:
%run "C:/Users/13496/geoplot.ipynb"

In [3]:
length=min_distances
profiles = pd.read_csv('D:\\18 Battlefield Road substation\\data_30min_avg_1.csv', header=0, decimal=',')
profiles = profiles.dropna()
profiles = profiles.reset_index(drop=True)
profiles=profiles.iloc[:, 1:]
header = profiles.columns.tolist()
profiles = profiles.astype('float64')
profiles=profiles/1000
ds = DFData(profiles)
profiles_q = pd.read_csv('D:\\18 Battlefield Road substation\\data_q_30min_avg_1.csv', header=0, decimal=',')
profiles_q = profiles_q.dropna()
profiles_q = profiles_q.reset_index(drop=True)
profiles_q=profiles_q.iloc[:, 1:]
header_q = profiles_q.columns.tolist()
profiles_q = profiles_q.astype('float64')
profiles_q=profiles_q/1000
ds_q = DFData(profiles_q)

In [4]:
def creat_net(length,p,q,geo_primary,geo_secondary):
    net = pp.create_empty_network()

    # Create HV and LV transformer buses
    hv_bus = pp.create_bus(net, name="HV Transformer Bus",geodata=geo_primary[0], vn_kv=33, type="n")
    lv_bus = pp.create_bus(net, name="LV Transformer Bus",geodata=geo_primary[0], vn_kv=11, type="n")

    # Create additional 20 LV buses
    lv_buses = [pp.create_bus(net, name=f'LV Bus {i}', vn_kv=11,geodata=geo_secondary[i],type='b') for i in range(0, 20)]

    # Create the transformer to step down voltage from 33kV to 11kV
    pp.create_transformer_from_parameters(net, hv_bus, lv_bus, sn_mva=20, vn_hv_kv=33, vn_lv_kv=11, vkr_percent=0.06,
                                      vk_percent=8, pfe_kw=0, i0_percent=0, name='HV-LV-Trafo')
    
    test_type = {"r_ohm_per_km": 0.128, "x_ohm_per_km": 0.37, "c_nf_per_km": 0.2, "max_i_ka": 0.257, "type": "cs"}
    pp.create_std_type(net, name="test_type", data=test_type, element="line")
    for n in range(0,20):
        from_bus = lv_bus
        to_bus =  pp.get_element_index(net, "bus", name='LV Bus %s'% n)
        pp.create_line(net, from_bus, to_bus, length_km=length[n]/1000,std_type="test_type",name='LV Line %s'% n)
        bus_idx = pp.get_element_index(net, "bus", name='LV Bus %s'% n)
        pp.create_load(net, bus_idx, p_mw=p[n]/1000, q_mvar=q[n]/1000,name='LV Load %s'% n)
    # Create a slack bus
    pp.create_ext_grid(net, hv_bus) 

    
    
    return net

In [5]:
net=creat_net(length,profiles.iloc[0], profiles_q.iloc[0],coordinates_pri,coordinates_sec)

In [7]:
def add_measurements(net,p,q):
    
    pp.create_measurement(net, "v", "bus", 1, 0.02, element=0) # Voltage magnitude at bus 0
    pp.create_measurement(net, "p", "bus", 0.1, 0.05, element=1) # Active power injection at bus 1
    pp.create_measurement(net, "q", "bus", 0.05, 0.02, element=1) # Reactive power injection at bus 1
    for i in range (0,20):
        for t in range(0,30):
            pp.create_measurement(net, "p", "line", p[i], 0.05, element=i, side="from") # Active power flow from bus 2 to bus 3
            pp.create_measurement(net, "q", "line", q[i], 0.02, element=i, side="from") # Reactive power flow from bus 2 to bus 3



In [11]:
vm_pu={}
va_degree={}
p_mw={}
for t in range(len(profiles)):
    print(f"Time step {t+1}")

    # Add measurements for the current time step
    add_measurements(net, profiles.iloc[t], profiles_q.iloc[t])

    # Perform state estimation
    success = estimate(net)
    vm_pu[t] = net.res_bus_est.vm_pu.tolist()
    p_mw[t] = net.res_bus_est.p_mw.tolist()
    va_degree[t] = net.res_bus_est.va_degree.tolist()
    print("State Estimation Successful: ", success)
    # Print estimated voltages and angles
    print("Estimated Voltages (p.u.):", net.res_bus_est.vm_pu)
    print("Estimated Voltage Angles (degree):", net.res_bus_est.va_degree)
    print()

Time step 1
State Estimation Successful:  True
Estimated Voltages (p.u.): 0     1.000000
1     0.996311
2     0.996308
3     0.996257
4     0.996251
5     0.996280
6     0.996209
7     0.996254
8     0.996235
9     0.996184
10    0.996196
11    0.996150
12    0.996245
13    0.996137
14    0.996257
15    0.996239
16    0.996198
17    0.996278
18    0.996078
19    0.996117
20    0.996221
21    0.996146
Name: vm_pu, dtype: float64
Estimated Voltage Angles (degree): 0     0.000000
1    -0.536616
2    -0.536888
3    -0.541186
4    -0.543874
5    -0.539464
6    -0.544582
7    -0.541679
8    -0.545296
9    -0.548867
10   -0.549525
11   -0.550276
12   -0.543320
13   -0.531474
14   -0.542010
15   -0.543345
16   -0.546478
17   -0.539300
18   -0.535320
19   -0.532766
20   -0.545143
21   -0.552623
Name: va_degree, dtype: float64

Time step 2
State Estimation Successful:  True
Estimated Voltages (p.u.): 0     1.000000
1     0.996345
2     0.996342
3     0.996291
4     0.996289
5     0.996311
6     

State Estimation Successful:  True
Estimated Voltages (p.u.): 0     1.000000
1     0.994848
2     0.994843
3     0.994795
4     0.994772
5     0.994758
6     0.994714
7     0.994746
8     0.994699
9     0.994663
10    0.994791
11    0.994658
12    0.994709
13    0.994669
14    0.994753
15    0.994772
16    0.994650
17    0.994765
18    0.994386
19    0.994419
20    0.994681
21    0.994621
Name: vm_pu, dtype: float64
Estimated Voltage Angles (degree): 0     0.000000
1    -0.857478
2    -0.858060
3    -0.862336
4    -0.867129
5    -0.868089
6    -0.869848
7    -0.868476
8    -0.874928
9    -0.878462
10   -0.862687
11   -0.877984
12   -0.874448
13   -0.852928
14   -0.868256
15   -0.866297
16   -0.879333
17   -0.867242
18   -0.851277
19   -0.849128
20   -0.876581
21   -0.880597
Name: va_degree, dtype: float64

Time step 12
State Estimation Successful:  True
Estimated Voltages (p.u.): 0     1.000000
1     0.994944
2     0.994939
3     0.994890
4     0.994871
5     0.994855
6     0.994821
7 

State Estimation Successful:  True
Estimated Voltages (p.u.): 0     1.000000
1     0.996311
2     0.996308
3     0.996267
4     0.996264
5     0.996256
6     0.996231
7     0.996260
8     0.996223
9     0.996208
10    0.996219
11    0.996167
12    0.996247
13    0.996193
14    0.996246
15    0.996271
16    0.996150
17    0.996276
18    0.996007
19    0.996023
20    0.996233
21    0.996151
Name: vm_pu, dtype: float64
Estimated Voltage Angles (degree): 0     0.000000
1    -0.517528
2    -0.517839
3    -0.521598
4    -0.523269
5    -0.524209
6    -0.523292
7    -0.522018
8    -0.527589
9    -0.527981
10   -0.527877
11   -0.531215
12   -0.523723
13   -0.514356
14   -0.525082
15   -0.521267
16   -0.533855
17   -0.520233
18   -0.511736
19   -0.510511
20   -0.524675
21   -0.532888
Name: va_degree, dtype: float64

Time step 22
State Estimation Successful:  True
Estimated Voltages (p.u.): 0     1.000000
1     0.996600
2     0.996598
3     0.996559
4     0.996557
5     0.996551
6     0.996525
7 

In [12]:
vm_pu = pd.DataFrame.from_dict(vm_pu)
va_degree = pd.DataFrame.from_dict(va_degree)
p_mw = pd.DataFrame.from_dict(p_mw)


In [14]:
vm_pu

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.996311,0.996345,0.996360,0.996376,0.996331,0.996152,0.995918,0.995791,0.995554,0.994926,...,0.996311,0.996600,0.996857,0.996968,0.997045,0.997089,0.996858,0.996853,0.996891,0.996912
2,0.996308,0.996342,0.996357,0.996373,0.996328,0.996149,0.995914,0.995787,0.995549,0.994921,...,0.996308,0.996598,0.996854,0.996965,0.997043,0.997087,0.996856,0.996851,0.996889,0.996910
3,0.996257,0.996291,0.996309,0.996325,0.996282,0.996109,0.995870,0.995745,0.995510,0.994873,...,0.996267,0.996559,0.996819,0.996933,0.997010,0.997053,0.996821,0.996816,0.996854,0.996875
4,0.996251,0.996289,0.996302,0.996319,0.996273,0.996092,0.995857,0.995725,0.995487,0.994850,...,0.996264,0.996557,0.996815,0.996929,0.997005,0.997047,0.996815,0.996810,0.996848,0.996868
5,0.996280,0.996311,0.996325,0.996339,0.996290,0.996102,0.995860,0.995726,0.995482,0.994837,...,0.996256,0.996551,0.996821,0.996935,0.997014,0.997056,0.996816,0.996812,0.996852,0.996874
6,0.996209,0.996240,0.996253,0.996271,0.996225,0.996044,0.995800,0.995677,0.995436,0.994796,...,0.996231,0.996525,0.996787,0.996903,0.996982,0.997028,0.996796,0.996793,0.996831,0.996850
7,0.996254,0.996294,0.996308,0.996324,0.996278,0.996093,0.995853,0.995713,0.995471,0.994829,...,0.996260,0.996552,0.996812,0.996922,0.997002,0.997044,0.996816,0.996811,0.996847,0.996866
8,0.996235,0.996263,0.996279,0.996293,0.996246,0.996064,0.995818,0.995678,0.995433,0.994778,...,0.996223,0.996519,0.996778,0.996895,0.996977,0.997020,0.996785,0.996777,0.996820,0.996846
9,0.996184,0.996204,0.996217,0.996239,0.996185,0.995993,0.995754,0.995640,0.995387,0.994740,...,0.996208,0.996504,0.996775,0.996885,0.996966,0.997015,0.996777,0.996772,0.996814,0.996838


In [50]:
net.bus

,name,vn_kv,type,zone,in_service
0,HV Transformer Bus,33.0,n,None,True
1,LV Transformer Bus,11.0,n,None,True
2,LV Bus 1,11.0,b,None,True
3,LV Bus 2,11.0,b,None,True
4,LV Bus 3,11.0,b,None,True
5,LV Bus 4,11.0,b,None,True
6,LV Bus 5,11.0,b,None,True
7,LV Bus 6,11.0,b,None,True
8,LV Bus 7,11.0,b,None,True
9,LV Bus 8,11.0,b,None,True


In [50]:
net.bus

,name,vn_kv,type,zone,in_service
0,HV Transformer Bus,33.0,n,None,True
1,LV Transformer Bus,11.0,n,None,True
2,LV Bus 1,11.0,b,None,True
3,LV Bus 2,11.0,b,None,True
4,LV Bus 3,11.0,b,None,True
5,LV Bus 4,11.0,b,None,True
6,LV Bus 5,11.0,b,None,True
7,LV Bus 6,11.0,b,None,True
8,LV Bus 7,11.0,b,None,True
9,LV Bus 8,11.0,b,None,True


In [15]:
vm_pu[1]

0     1.000000
1     0.996345
2     0.996342
3     0.996291
4     0.996289
5     0.996311
6     0.996240
7     0.996294
8     0.996263
9     0.996204
10    0.996239
11    0.996182
12    0.996269
13    0.996173
14    0.996287
15    0.996267
16    0.996221
17    0.996315
18    0.996111
19    0.996173
20    0.996264
21    0.996174
Name: 1, dtype: float64